In [32]:
import requests
from newspaper import Article
import pandas as pd
from time import sleep
import os 
from slugify import slugify
import ast
import numpy as np



In [42]:
import spacy
nlp = spacy.load("en_core_web_md")



In [22]:
df = pd.read_csv('../BLM-GFAABT.csv')



In [27]:
def str_2_list(links):
    try:
        return ast.literal_eval(links)
    except:
        return links

    
def good(url):
    if 'twitter.com' in url:
        return False
    elif 'facebook.com' in url:
        return False
    return True

df['urls'] = df['urls'].apply(str_2_list)
urls = list(set([item for sublist in df['urls'].values for item in sublist]))



urls = [u for u in urls if good(u) == True]
len(urls)

5606

In [28]:
def get_html(url):
    """Attempt to open a file based on the url."""
    location = locate(url)

    with open(location, "r") as infile:
        html = infile.read()
    return html

def locate(url):
    """Create file name and place in directory"""
    file_name = slugify(url)
    location = os.path.join("blm-html", file_name)
    return location

def get_article_info(url):
    """Load and parse a newspaper url."""
    html = get_html(url)
    article = Article(url)
    article.html = html
    article.download_state = 2
    article.parse()

    article_details = {
        "title": article.title,
        "text": article.text,
        "url": article.url,
        "authors": article.authors,
        "html": article.html,
        "date": article.publish_date,
        "description": article.meta_description,
    }
    return article_details

In [33]:
parsed = []
for url in list(urls):
    try:
        parsed.append(get_article_info(url))
    except (FileNotFoundError, OSError):
        pass
    

In [36]:
df = pd.DataFrame(parsed)
len(df)

3082

In [47]:
import re

def extract_size(text):
    
    # shorten text to avoid later discussion of other protests stuff
    doc = nlp(text)
    text = ' '.join([s.text for s in list(doc.sents)][:5])
    
    s = text.lower()
    for prefix in ['up to', 'more than','around','at least', 'about', 'approximately', 'over', 'almost']:
        for suffix in ['protesters', 'people', 'town residents']:
            h = re.findall('%s \d+(?:,\d+)? %s' % (prefix, suffix), s)
            if len(h) > 0:
                return h[0]
    if 'hundreds of protesters' in s:
        return 'hundreds'
    if 'hundreds of people' in s:
        return 'hundreds'
    if 'few hundred people' in s:
        return 'hundreds'
    if 'few hundred people' in s:
        return 'hundreds'
    if 'hundreds of protestors' in s:
        return 'hundreds'
    if 'over a hundred' in s:
        return 'hundred'

    for d in ['dozens of protestors', 'few dozen' , 'at least a dozen']:
        if d in s:
            return d

    
    size = []
    
    verbs = ['protested', 'marched', 'demonstrated', 'gathered', 'attended' , 'participated', 'formed']
    for chunk in doc.noun_chunks:
        if chunk.root.head.text in verbs:
            size.append([chunk.text + ' ' +                    chunk.root.head.text])

    return size 



In [51]:
df['text'].sample(20).fillna('').apply(extract_size)

2447                                                   []
941     [[Demonstrators gathered], [Crowds gathered], ...
2493                                                   []
1708    [[the crowd gathered], [The crowd marched], [t...
1755    [[protesters marched], [more than two miles ma...
1157         [[Anti-police brutality activists gathered]]
1215                                                   []
1401                                 more than 150 people
458                                                    []
701                                                    []
561                                                    []
1689                                             hundreds
524                                [[Protesters marched]]
3061                                                   []
2844                                                   []
1485                                  more than 30 people
389                                                    []
2922          